In [23]:
#Importing library
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

np.random.seed(1000)

#Instantiation
AlexNet = Sequential()

#1st Convolutional Layer
AlexNet.add(Conv2D(filters=96, input_shape=(32,32,3), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096, input_shape=(32,32,3,)))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.4))

#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#3rd Fully Connected Layer
AlexNet.add(Dense(1000))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#Output Layer
AlexNet.add(Dense(20))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('softmax'))

#Model Summary
AlexNet.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 8, 8, 96)          34944     
_________________________________________________________________
batch_normalization_9 (Batch (None, 8, 8, 96)          384       
_________________________________________________________________
activation_9 (Activation)    (None, 8, 8, 96)          0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 96)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 4, 4, 256)         614656    
_________________________________________________________________
batch_normalization_10 (Batc (None, 4, 4, 256)         1024      
_________________________________________________________________
activation_10 (Activation)   (None, 4, 4, 256)        

In [24]:
# Compiling the model
AlexNet.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])


In [35]:
import numpy as np

images = np.load('trnImage.npy')
labels = np.load('trnLabel_coarse.npy')
labels_fine = np.load('trnLabel_fine.npy')

test_images = np.load('tstImage.npy')
test_labels = np.load('tstLabel_coarse.npy')
test_labels_fine = np.load('tstLabel_fine.npy')

print(f'Images Shape: {images.shape}')
print(f'Train Labels Coarse Shape: {labels.shape}')

Images Shape: (32, 32, 3, 50000)
Train Labels Coarse Shape: (50000,)


In [36]:
train_images_reshaped = np.transpose(images, (3, 0, 1, 2))
test_images_reshaped = np.transpose(test_images, (3, 0, 1, 2))
labels_reshaped = labels.reshape(50000, 1)

labels_fine_reshaped = labels_fine.reshape(50000, 1)


x_train = train_images_reshaped
#y_train = labels_reshaped
y_train = labels_fine_reshaped

x_test = test_images_reshaped
y_test = test_labels

#Train-validation-test split
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.3)


#Onehot Encoding the labels.
from sklearn.utils.multiclass import unique_labels
from keras.utils import to_categorical

#Since we have 20 classes we should expect the shape[1] of y_train,y_val and y_test to change from 1 to 20
y_train=to_categorical(y_train)
y_val=to_categorical(y_val)
y_test=to_categorical(y_test)

#Verifying the dimension after one hot encoding
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

#Image Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1 )
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip= True,zoom_range=.1)

#Fitting the augmentation defined above to the data
train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)


((35000, 32, 32, 3), (35000, 100))
((15000, 32, 32, 3), (15000, 100))
((10000, 32, 32, 3), (10000, 20))


In [27]:
print(f'Images Shape: {train_images_reshaped.shape}')
print(f'Train Labels Coarse Shape: {test_images_reshaped.shape}')

Images Shape: (50000, 32, 32, 3)
Train Labels Coarse Shape: (10000, 32, 32, 3)


In [37]:
#Defining the parameters
batch_size= 100
epochs=100
learn_rate=.001

In [38]:
print(x_train.shape[0])

35000


In [ ]:
AlexNet = keras.models.load_model('alexNet_coarse')

In [31]:
#Training the model
AlexNet.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs = epochs, steps_per_epoch = x_train.shape[0]//batch_size, validation_data = val_generator.flow(x_val, y_val, batch_size=batch_size), validation_steps = len(x_test)//batch_size, verbose=1)


Epoch 1/100
350/350 [==============================] - 156s 442ms/step - loss: 2.6588 - accuracy: 0.1843 - val_loss: 3.0741 - val_accuracy: 0.1523
Epoch 2/100
350/350 [==============================] - 159s 455ms/step - loss: 2.3056 - accuracy: 0.2861 - val_loss: 3.1576 - val_accuracy: 0.1925
Epoch 3/100
350/350 [==============================] - 161s 461ms/step - loss: 2.1845 - accuracy: 0.3224 - val_loss: 2.4502 - val_accuracy: 0.2684
Epoch 4/100
350/350 [==============================] - 154s 441ms/step - loss: 2.0883 - accuracy: 0.3517 - val_loss: 2.7001 - val_accuracy: 0.2463
Epoch 5/100
350/350 [==============================] - 147s 420ms/step - loss: 2.0153 - accuracy: 0.3740 - val_loss: 2.2482 - val_accuracy: 0.3184
Epoch 6/100
350/350 [==============================] - 148s 422ms/step - loss: 1.9406 - accuracy: 0.3971 - val_loss: 3.1157 - val_accuracy: 0.2049
Epoch 7/100
350/350 [==============================] - 148s 423ms/step - loss: 1.8967 - accuracy: 0.4132 - val_loss: 2

350/350 [==============================] - 153s 436ms/step - loss: 0.4478 - accuracy: 0.8606 - val_loss: 2.3500 - val_accuracy: 0.4586
Epoch 57/100
350/350 [==============================] - 154s 439ms/step - loss: 0.4257 - accuracy: 0.8667 - val_loss: 2.6994 - val_accuracy: 0.4104
Epoch 58/100
350/350 [==============================] - 154s 440ms/step - loss: 0.4139 - accuracy: 0.8722 - val_loss: 2.3925 - val_accuracy: 0.4527
Epoch 59/100
350/350 [==============================] - 154s 439ms/step - loss: 0.3917 - accuracy: 0.8778 - val_loss: 2.6760 - val_accuracy: 0.4070
Epoch 60/100
350/350 [==============================] - 153s 438ms/step - loss: 0.4041 - accuracy: 0.8734 - val_loss: 2.3306 - val_accuracy: 0.4711
Epoch 61/100
350/350 [==============================] - 152s 433ms/step - loss: 0.3915 - accuracy: 0.8779 - val_loss: 2.4518 - val_accuracy: 0.4429
Epoch 62/100
350/350 [==============================] - 154s 441ms/step - loss: 0.3888 - accuracy: 0.8797 - val_loss: 2.6027 

In [32]:
AlexNet.save('alexNet_coarse')

INFO:tensorflow:Assets written to: alexNet_coarse\assets


In [40]:
#Importing library
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

np.random.seed(1000)

#Instantiation
AlexNetFine = Sequential()

#1st Convolutional Layer
AlexNetFine.add(Conv2D(filters=96, input_shape=(32,32,3), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNetFine.add(BatchNormalization())
AlexNetFine.add(Activation('relu'))
AlexNetFine.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNetFine.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNetFine.add(BatchNormalization())
AlexNetFine.add(Activation('relu'))
AlexNetFine.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNetFine.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNetFine.add(BatchNormalization())
AlexNetFine.add(Activation('relu'))

#4th Convolutional Layer
AlexNetFine.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNetFine.add(BatchNormalization())
AlexNetFine.add(Activation('relu'))

#5th Convolutional Layer
AlexNetFine.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNetFine.add(BatchNormalization())
AlexNetFine.add(Activation('relu'))
AlexNetFine.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNetFine.add(Flatten())
# 1st Fully Connected Layer
AlexNetFine.add(Dense(4096, input_shape=(32,32,3,)))
AlexNetFine.add(BatchNormalization())
AlexNetFine.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNetFine.add(Dropout(0.4))

#2nd Fully Connected Layer
AlexNetFine.add(Dense(4096))
AlexNetFine.add(BatchNormalization())
AlexNetFine.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#3rd Fully Connected Layer
AlexNetFine.add(Dense(1000))
AlexNetFine.add(BatchNormalization())
AlexNetFine.add(Activation('relu'))
#Add Dropout
AlexNetFine.add(Dropout(0.4))

#Output Layer
AlexNetFine.add(Dense(100))
AlexNetFine.add(BatchNormalization())
AlexNetFine.add(Activation('softmax'))

#Model Summary
AlexNetFine.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 8, 8, 96)          34944     
_________________________________________________________________
batch_normalization_27 (Batc (None, 8, 8, 96)          384       
_________________________________________________________________
activation_27 (Activation)   (None, 8, 8, 96)          0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 4, 4, 96)          0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 4, 4, 256)         614656    
_________________________________________________________________
batch_normalization_28 (Batc (None, 4, 4, 256)         1024      
_________________________________________________________________
activation_28 (Activation)   (None, 4, 4, 256)        

In [ ]:
# Compiling the model
AlexNetFine.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])


In [ ]:
AlexNetFin.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs = epochs, steps_per_epoch = x_train.shape[0]//batch_size, validation_data = val_generator.flow(x_val, y_val, batch_size=batch_size), validation_steps = len(x_test)//batch_size, verbose=1)
